In [ ]:
import cv2
import mediapipe as mp 
import numpy as np
import os
import base64
import numpy as np
from clickhouse_driver import Client
from deepface import DeepFace
from datetime import datetime

# Fungsi untuk membuat folder temporary_photo
def create_temporary_folder():
    folder_path = "temporary_photo"
    
    # Cek apakah folder temporary_photo sudah ada
    if os.path.exists(folder_path):
        # Jika sudah ada, hapus semua file di dalamnya
        [os.remove(os.path.join(folder_path, file)) for file in os.listdir(folder_path)]
    else:
        # Jika belum ada, buat folder temporary_photo
        os.makedirs(folder_path)

# Fungsi untuk mendeteksi posisi kepala dalam gambar
def detect_head_position(results, frame, img_w, img_h):
    # Inisialisasi list untuk menyimpan landmark wajah dalam 2D dan 3D
    face_3d = []
    face_2d = []
    # Inisialisasi threshold untuk deteksi blur, kegelapan, dan kecerahan
    tresshold_blur = 0
    tresshold_dark = 80
    tresshold_bright = 140
    # Konversi gambar ke grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Hitung nilai laplacian dan rata-rata pixel gambar
    laplacian = np.var(cv2.Laplacian(gray, cv2.CV_64F))
    average_pixel_value = np.mean(gray)

    # Periksa apakah terdeteksi wajah dalam gambar
    if results.multi_face_landmarks is not None and len(results.multi_face_landmarks) > 0:
        # Periksa kondisi untuk deteksi blur, kegelapan, dan kecerahan
        if average_pixel_value < tresshold_bright and tresshold_dark < average_pixel_value and tresshold_blur < laplacian:
            for face_landmarks in results.multi_face_landmarks:
                for idx, lm in enumerate(face_landmarks.landmark):
                    if idx in [1, 33, 263, 62, 291, 199]:
                        x, y = int(lm.x * img_w), int(lm.y * img_h)
                        face_2d.append([x, y])
                        face_3d.append([x, y, lm.z])
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)

            # Hitung parameter kamera dan rotasi wajah
            focal_length = 1 * img_w
            cam_matrix = np.array([[focal_length, 0, img_w/2],
                                   [0, focal_length, img_h/2],
                                   [0, 0, 1]])
            dist_coeffs = np.zeros((4,1))
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_coeffs)
            rmat, _ = cv2.Rodrigues(rot_vec)
            angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
            x, y, z = angles[0] * 360, angles[1] * 360, angles[2] * 360

            # Tentukan posisi kepala berdasarkan sudut rotasi
            conditions = {
                "looking_forward": (-10 < x < 10 and -10 < y < 10 and -5 < z < 5, r".\temporary_photo\looking_forward.jpg"),
                "looking_tilty_left": (x < 5 and x > -5 and y < -10 and y > -25 and z < 5 and z > -5, r".\temporary_photo\looking_tilty_left.jpg"),
                "looking_left": (x < 5 and x > -5  and y < -25 and z < 5 and z > -5, r".\temporary_photo\looking_left.jpg"),
                "looking_tilty_right": (x < 5 and x > -5 and y > 10 and y < 25 and z < 5 and z > -5, r".\temporary_photo\looking_tilty_right.jpg"),
                "looking_right": (x < 5 and x > -5  and y > 25 and z < 5 and z > -5, r".\temporary_photo\looking_right.jpg"),
                "looking_down": (x < -10 and y < 8 and y > -8, r".\temporary_photo\looking_down.jpg"),
                "looking_down_tilty_left": (x < -5 and y < -8 and z < 5 and z > -5, r".\temporary_photo\looking_down_tilty_left.jpg"),
                "looking_down_tilty_right": (x < -5 and y > 8 and z < 5 and z > -5, r".\temporary_photo\looking_down_tilty_right.jpg")
            }

            # Simpan gambar sesuai dengan posisi kepala yang terdeteksi
            for text, (condition, save_path) in conditions.items():
                if condition:
                    if not os.path.exists(save_path):
                        cv2.imwrite(save_path, frame)
                    break

            # Periksa apakah sudah terdapat 8 gambar yang tersimpan
            folder_path = "temporary_photo"
            image_files = [(filename, cv2.imread(os.path.join(folder_path, filename))) for filename in os.listdir(folder_path) if filename.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
            if len(image_files) == 8 :
                return True

    return False

# Fungsi untuk membuat folder transformed_images
def create_transformed_folder():
    folder_path = "transformed_images"
    
    # Cek apakah folder transformed_images sudah ada
    if os.path.exists(folder_path):
        # Jika sudah ada, hapus semua file di dalamnya
        [os.remove(os.path.join(folder_path, file)) for file in os.listdir(folder_path)]
    else:
        # Jika belum ada, buat folder transformed_images
        os.makedirs(folder_path)

# Fungsi untuk menerapkan transformasi ke gambar-gambar dalam folder temporary_photo
def apply_transformations_to_temporary_images(temporary_photo_dir):
    # Tentukan direktori output
    output_dir = "transformed_images"
    os.makedirs(output_dir, exist_ok=True)

    # Definisikan transformasi yang akan dilakukan
    transformations = [
        ("brighten", lambda img: cv2.convertScaleAbs(img, beta=50)),
        ("darken", lambda img: cv2.convertScaleAbs(img, beta=-50)),
        ("high_contrast", lambda img: cv2.convertScaleAbs(img, alpha=1.5, beta=0)),
        ("low_contrast", lambda img: cv2.convertScaleAbs(img, alpha=0.6, beta=0)),
        ("blur", lambda img: cv2.GaussianBlur(img, (15, 15), 0)),
        ("sharpen", lambda img: cv2.filter2D(img, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))),
        ("add_noise", lambda img: cv2.add(img, np.random.normal(0, 0.5, img.shape).astype(np.uint8))),
        ("rotate_only", lambda img: img)
    ]

    # Tentukan sudut rotasi
    rotation_angles = [0, 15, 30, 45, 60, 75, 285, 300, 315, 330, 345]

    # Ambil daftar file gambar di dalam direktori temporary_photo
    image_files = [filename for filename in os.listdir(temporary_photo_dir) if filename.endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    # Terapkan transformasi ke setiap gambar di dalam direktori temporary_photo
    for image_file in image_files:
        image_path = os.path.join(temporary_photo_dir, image_file)
        image = cv2.imread(image_path)

        # Iterasi melalui setiap transformasi dan sudut rotasi
        for name, transform_func in transformations:
            for angle in rotation_angles:
                # Putar gambar
                rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1), (image.shape[1], image.shape[0]))

                # Terapkan transformasi
                transformed_image = transform_func(rotated_image.copy())

                # Simpan gambar yang telah ditransformasi
                output_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_rotated_{angle}_{name}.jpg")
                cv2.imwrite(output_path, transformed_image)

    print("Transformations and rotations completed, and images saved successfully.")

# Fungsi untuk menangkap wajah dari kamera dan menyimpannya
def capture_face():
    # Buat folder temporary_photo
    create_temporary_folder()

    # Inisialisasi model deteksi wajah dari MediaPipe
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Buka kamera
    cap =  cv2.VideoCapture(0)
    while cap.isOpened():
        # Baca frame dari kamera
        success, frame = cap.read()

        if not success:
            print("Error: Could not read frame.")
            break

        # Konversi frame ke RGB
        frame = cv2.cvtColor(cv2.flip(frame,1), cv2.COLOR_BGR2RGB)
        frame.flags.writeable = False

        # Proses deteksi wajah menggunakan MediaPipe
        results = face_mesh.process(frame)
        frame.flags.writeable = True

        # Konversi frame kembali ke BGR
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        img_h, img_w, _ = frame.shape

        # Deteksi posisi kepala
        if detect_head_position(results, frame, img_w, img_h):
            cap.release()
            break
        else:
            # Jika tidak terdeteksi wajah, tampilkan pesan
            if results.multi_face_landmarks is None:
                cv2.putText(frame, "No face detected", (20, 420), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

        # Tampilkan frame dengan posisi kepala yang terdeteksi
        cv2.imshow("Head Position Estimation", frame)
        # Jika tombol 'q' ditekan, simpan frame sebagai lol.jpg
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.imwrite("lol.jpg", frame)
            break

    # Buat folder transformed_images dan terapkan transformasi ke gambar-gambar dalam temporary_photo
    create_transformed_folder()
    apply_transformations_to_temporary_images("temporary_photo")

    # Tutup kamera dan jendela OpenCV
    cap.release()
    cv2.destroyAllWindows()

# Fungsi untuk mengubah gambar menjadi vektor representasi dan base64
def image_to_vector(img_path):
    try:
        # Gunakan DeepFace untuk mendapatkan vektor representasi dan base64
        result = DeepFace.represent(img_path, 
                                    model_name = "Facenet512",
                                    enforce_detection = False,
                                    detector_backend = "fastmtcnn")
        
        # Ambil nilai dari kunci pertama
        vector_representation = result[0]['embedding']

        # Ubah img_path ke base64
        with open(img_path, "rb") as image_file:
            base64_encoded = base64.b64encode(image_file.read()).decode('utf-8')

        return vector_representation, base64_encoded
    except Exception as e:
        print("Error:", str(e))
        return None, None
    pass

# Fungsi untuk menyimpan data karyawan ke tabel ClickHouse
def save_employee_data(employee_id, employee_name, vector_representation, base64_encoded):
    try:
        # Inisialisasi koneksi ClickHouse
        client = Client(host='localhost', port=9000, user='default', database='default')
        
        # Query untuk menyimpan data ke tabel karyawan
        query = f"INSERT INTO employee (employee_id, employee_name, embedding, image_base64) VALUES ({employee_id}, '{employee_name}', {vector_representation}, '{base64_encoded}');"

        # Eksekusi query
        client.execute(query)
    except Exception as e:
        print(f"Error: {e}")
    pass

# Fungsi untuk memasukkan data karyawan ke database
def insert_ke_database():
    # Direktori tempat Anda menyimpan 20 foto
    input_directory = './transformed_images'
    employee_name = input("Masukkan nama karyawan: ")
    employee_id = int(input("Masukkan ID karyawan: "))    

    # Menangkap wajah dari kamera dan menyimpannya
    capture_face()

    # Mengecek apakah direktori ada dan tidak kosong
    if os.path.exists(input_directory) and os.listdir(input_directory):
        # Iterasi setiap file dalam direktori
        for filename in os.listdir(input_directory):
            if filename.endswith(".jpg"):
                # Path lengkap ke file
                img_path = os.path.join(input_directory, filename)

                # Menggunakan fungsi image_to_vector untuk mendapatkan vektor dan base64
                vector_representation, base64_encoded = image_to_vector(img_path)

                # Menggunakan fungsi save_employee_data untuk menyimpan data ke ClickHouse
                if vector_representation is not None and base64_encoded is not None:
                    save_employee_data(employee_id, employee_name, vector_representation, base64_encoded)
                    print(f"{employee_name} - {filename} berhasil dimasukkan ke database.")
    else:
       print("Direktori kosong atau tidak ditemukan.")

# Panggil fungsi untuk memasukkan data karyawan ke database
insert_ke_database()
